In [2]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 4.1 MB/s 
     |████████████████████████████████| 4.2 MB 30.9 MB/s 
     |████████████████████████████████| 1.2 MB 47.0 MB/s 
     |████████████████████████████████| 84 kB 4.0 MB/s 
     |████████████████████████████████| 6.6 MB 46.2 MB/s 
     |████████████████████████████████| 596 kB 53.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=8725f59bed113af0291613ccd82804d9488a014d664ed2fc6faa51bb6dfa2595
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

data = pd.read_pickle('/content/drive/Shareddrives/TCC 2022 - Marcelo/lp - paulo/data_v0_embeddings.pkl')

In [7]:
from sentence_transformers import SentenceTransformer
bert_extractor = SentenceTransformer('distiluse-base-multilingual-cased-v1')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/452 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

In [8]:
data.fillna('',inplace=True)

In [9]:
c = ['what','when','where','who','why','how']
for item in c:
  print('Gerando embeddings para',item)
  emb = bert_extractor.encode(data[item].to_list())
  data[item+'_embedding'] = list(emb)

Gerando embeddings para what
Gerando embeddings para when
Gerando embeddings para where
Gerando embeddings para who
Gerando embeddings para why
Gerando embeddings para how


In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer


def is_stopword_component(component_text, stop_words = set(stopwords.words('english'))):
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(component_text)

    for word in words:
        if word.lower() not in stop_words:
            return False

    return True

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [25]:
# Grafo inicial pode ser um 3-NN entre eventos

from sklearn.neighbors import kneighbors_graph
import networkx as nx
import numpy as np

A = kneighbors_graph(np.array(data['embedding'].to_list()), n_neighbors = 3, n_jobs = -1, metric='cosine')

G = nx.Graph(A)

In [26]:
# Percorrendo a rede, adicionando metadados e as componentes

counter = 0
for index,row in data.iterrows():
  G.nodes[counter]['type']='event'
  G.nodes[counter]['text']=row['text']
  G.nodes[counter]['features']=row['embedding']
  G.nodes[counter]['id'] = row['_id']

  #print(row)
  for item in c:

    if item == 'text': continue
    if row[item]=='': continue
    if is_stopword_component(row[item]): continue

    G.add_edge(counter,row[item])
    G.nodes[row[item]]['type']=item
    G.nodes[row[item]]['text']=row[item]
    G.nodes[row[item]]['features']=row[item+'_embedding']
    if item == 'where':
        G.nodes[row[item]]['coords'] = row['coords']

  counter += 1

In [27]:
import pickle
with open("/content/drive/Shareddrives/TCC 2022 - Marcelo/lp - paulo/network.pkl", 'wb') as f:
    pickle.dump(G, f)

In [28]:
with open("/content/drive/Shareddrives/TCC 2022 - Marcelo/lp - paulo/network.pkl", 'rb') as f:
    G_test = pickle.load(f)

In [31]:
for node in G_test.nodes:
    if G_test.nodes[node]['type'] == 'where':
        print(G_test.nodes[node]['coords'])

A saída de streaming foi truncada nas últimas 5000 linhas.
(29.9722976, -90.0653803)
(78.6547604, 16.328362908786612)
(50.6310504, -3.329984389371436)
(33.950150449999995, -98.55572885692527)
(52.4833326, -2.0674067)
(37.2577171, -120.9988154)
(39.7465856, -75.53974819022754)
(32.7522877, -79.87462531951533)
(37.7360416, -122.370526)
(40.6976019, -74.2632024)
(25.7971515, -80.2793795)
(40.687547, -73.9973352)
(37.7885108, -122.3882669)
(38.0049214, -121.805789)
(25.9165027, -97.4558301)
(28.1219416, -16.7241893)
(33.7382088, -81.0978734)
(41.86636555, -87.60662585681982)
(42.2781401, -74.9159946)
(17.994037499999997, -76.78402706283784)
(34.6423117, 50.8800969)
(33.7680065, 66.2385139)
(36.6981042, 137.862068)
(35.6536411, 139.6905386)
(34.4216681, -119.70002086811618)
(31.8819798, -112.8153361)
(36.1023552, 114.33633)
(32.701938999999996, -97.10562379033699)
(-45.0321923, 168.661)
(39.327894, -74.5034858)
(32.1686097, 35.2337655)
(44.056258, -79.461708)
(32.9131295, -80.0629981965219)